# Set Variables

In [1]:
# -------- method: first
# which_method = "first"
which_method = "second"

if which_method == "second":
    top_k = 20


# -------- method_way
method_way = "c" # a, b, c

if (method_way == "a" and which_method == "second"):
    bug_localization_accuracy = [1] # main data - th

if method_way == "b":
    bug_localization_accuracy = [1] # main data - th

if method_way == "c":
    bug_localization_accuracy = [0.4, 0.5, 0.6, 0.7, 0.8]


# -------- dataset
# software_name = "camel"
# software_name = "cloudstack"
software_name = "geode"
# software_name = "HBase"


# -------- google colab
# on_google_colab = True
on_google_colab = False

# Google Colab

In [2]:
if software_name == "camel":
    dataset_file_name = "dataset_camel - v.01"
    dataset_file_name_ts = "timeserAll_camel - v.01"
    
elif software_name == "cloudstack":
    dataset_file_name = "cloudstack_bugs - v.01"
    dataset_file_name_ts = "cloudstack_TimeSeries - v.01"
    
elif software_name == "geode":
    dataset_file_name = "geode_Bug - v.01"
    dataset_file_name_ts = "geode_TS - v.01"
    
else:
    dataset_file_name = "dataset_hbase - v.03"
    dataset_file_name_ts = "timeserAll - v.03"

In [3]:
if on_google_colab:
    
    !pip install enlighten
    !pip install --upgrade statsmodels 
    !pip install --upgrade matplotlib
    
    # load data from google drive
    from google.colab import drive
    drive.mount("/content/gdrive", force_remount=True)
    !ls "/content/gdrive/My Drive/"
    
    tempPre = "gdrive/MyDrive/Colab Notebooks/paper/Indirect Method/"
    pre_path_data = tempPre + "data/" + software_name + "/"
    
    # you can choose one of these and comment out the other:
    # pre_path_output = tempPre + "output/"
    pre_path_output = tempPre + "output/" + dataset_file_name + "/" + which_method + "/"

else:
    tempPre =    "E:/darsy/00/02- arshad/10- paper code/"
    tempData =   "00- My Data/Datasets/Indirect Method/"
    tempOutput = "01- Jupyter Notebook/InDirect Method/00. Output/"
    pre_path_data =   tempPre + tempData   + software_name + "/"
    pre_path_output = tempPre + tempOutput + software_name + "/" + dataset_file_name + "/" + which_method + "/"

# Libs

In [4]:
import random

import pandas as pd

# Config

In [5]:
mypaths = {
    "dataset": pre_path_data + dataset_file_name + ".csv"
}

preprocessing_params = {
    "dataset": {
        # time_stamp is the time that we change the class for a bug fixing.
        # id stands for identifier
        "columns_name":   ["bug_report_id", "class_id", "time_stamp", "label"],
        "columns_dtype" : {0: "int64", 1: "int64", 2: "int64", 3:"int32"}
    }
}

if which_method == "second":
    preprocessing_params["top_k"] = top_k

In [6]:
bug_localization_accuracy

[0.4, 0.5, 0.6, 0.7, 0.8]

# Read Files

In [7]:
df_main = pd.read_csv(
    mypaths["dataset"], 
    names=preprocessing_params["dataset"]["columns_name"], 
    dtype=preprocessing_params["dataset"]["columns_dtype"], 
    header=None
)

# MyDataset

In [8]:
class MyDatasetFirst():
    
    def __init__(self, df_main, bug_localization_accuracy):
        self.df_main = df_main.copy()
        self.bla = bug_localization_accuracy
        
        self.df_data = {} # [df1-bla[0], df2-bla[1], ...]
        self.cIDs = None
        self.brIDs_num_ft = {} # {"0.50": {"true": 400, "false":50}, ...}
        self.brIDs_num_total = None
        
        # ----------------- Read File & initialize brIDs_num_ft
        for bla in bug_localization_accuracy:
            tempDF = df_main.copy()
            self.df_data[bla] = tempDF
            
            self.brIDs_num_ft[bla] = {
                "true": None,
                "false": None
            }
            
        
        self.cIDs = set(self.df_main["class_id"].to_list()) # cIDs: class ids
    
    
    # brids: bug report ids
    # cids: class ids
    def select_brids_replace_cids(self):
        
        # -------- select_bug_reports
        # randomly select bug reports whose class ids will be replaced
        dfg_brIDs = self.df_main.groupby("bug_report_id") # brIDs: bug report ids
        self.brIDs_num_total = len(dfg_brIDs) # number of bug reports
        brIDs = list(dfg_brIDs.groups.keys())

        for bla in self.bla:
            bla_false = 1 - bla
            self.brIDs_num_ft[bla]["false"] = round(self.brIDs_num_total * bla_false)
            self.brIDs_num_ft[bla]["true"] = self.brIDs_num_total - self.brIDs_num_ft[bla]["false"]
            # selects elements without replacement
            brIDs_false = random.sample(brIDs, self.brIDs_num_ft[bla]["false"])
            
            df_brIDs_false = dfg_brIDs.filter(lambda g: brIDs_false.count(g.name) > 0)
            
            # -------- replace_class_ids
            df_brIDs_false.groupby("bug_report_id").apply(lambda g: self.replacement_false_(g, bla))
    
    
    def replacement_false_(self, g, bla):
        cIDs_true = set(g["class_id"].to_list()) # t_cids: true  cids
        cIDs_false = list(self.cIDs - cIDs_true)  # f_cids: false cids
        # selects elements that can be duplicated
        cIDs_false_selected = random.choices(cIDs_false, k=len(g))
        self.df_data[bla].loc[g.index, "class_id"] = cIDs_false_selected
    
    
    def free_memory(self):
        self.df_main = None
        self.cIDs = None

In [9]:
class MyDatasetSecond():
    
    def __init__(self, df_main, columns_name, columns_dtype, bug_localization_accuracy, top_k=5):
        self.df_main = df_main.copy()
        self.bla = bug_localization_accuracy
        self.top_k = top_k
        
        self.cIDs = set(self.df_main["class_id"].to_list()) # cIDs: class ids
        
        self.df_data = {} # [df1, df2, ...]
        self.brIDs_num_ft = {} # {"0.50": {"true": 400, "false":50}, ...}
        self.brIDs_num_total = None
        
        for bla in self.bla:
            temp = {}
            for cn, cd in zip(columns_name, columns_dtype.values()):
                temp[cn] = pd.Series(dtype=cd)
            tempEmptyDF = pd.DataFrame(temp)
            self.df_data[bla] = tempEmptyDF
            
            self.brIDs_num_ft[bla] = {
                "true": None,
                "false": None
            }
    
    
    # brids: bug report ids
    # cids: class ids
    def select_brids_replace_cids(self):
        
        # -------- select_bug_reports
        # randomly select bug reports whose class ids will be replaced
        dfg_brIDs = self.df_main.groupby("bug_report_id") # brIDs: bug report ids
        self.brIDs_num_total = len(dfg_brIDs) # number of bug reports
        brIDs = list(dfg_brIDs.groups.keys())
        
        for i, bla in enumerate(self.bla):
            bla_false = 1 - bla
            self.brIDs_num_ft[bla]["false"] = round(self.brIDs_num_total * bla_false)
            self.brIDs_num_ft[bla]["true"] = self.brIDs_num_total - self.brIDs_num_ft[bla]["false"]
            # random.sample: selects elements without replacement
            brIDs_false = random.sample(brIDs, self.brIDs_num_ft[bla]["false"])
            
            df_brIDs_false = dfg_brIDs.filter(lambda g: brIDs_false.count(g.name) > 0)
            df_brIDs_false.groupby("bug_report_id").apply(lambda g: self.replacement_false_(g, bla))
            
            df_brIDs_true = dfg_brIDs.filter(lambda g: brIDs_false.count(g.name) == 0)
            df_brIDs_true.groupby("bug_report_id").apply(lambda g: self.replacement_true_(g, bla))
    
    
    def replacement_false_(self, g, bla):
        bug_cids_true = g["class_id"].to_list()
        num_bug_cids_true = len(bug_cids_true)
        bug_cids_true_set = set(bug_cids_true)
        bug_cids_false = list(self.cIDs - bug_cids_true_set)
        if num_bug_cids_true >= self.top_k:
            # random.choices: selects elements that can be duplicated
            bug_cids_false_selected = random.choices(bug_cids_false, k=self.top_k)
            bug_cids_index = random.sample(list(range(num_bug_cids_true)), k=self.top_k)
            temp = self.df_main.loc[g.index[bug_cids_index], :].copy()
            temp["class_id"] = bug_cids_false_selected
        else:
            bug_cids_false_selected = random.choices(bug_cids_false, k=num_bug_cids_true)
            temp = self.df_main.loc[g.index, :].copy()
            temp["class_id"] = bug_cids_false_selected
            
        self.df_data[bla] = self.df_data[bla].append(temp, ignore_index = True)
    
    
    def replacement_true_(self, g, bla):
        bug_cids_true = g["class_id"].to_list()
        num_bug_cids_true = len(bug_cids_true)
        if num_bug_cids_true >= self.top_k:
            bug_cids_index = random.sample(list(range(num_bug_cids_true)), k=self.top_k)
            temp = self.df_main.loc[g.index[bug_cids_index], :].copy()
        else:
            temp = self.df_main.loc[g.index, :].copy()
            
        self.df_data[bla] = self.df_data[bla].append(temp, ignore_index = True)
    
    def free_memory(self):
        self.df_main = None
        self.cIDs = None

## obj

In [10]:
if which_method == "first":
    print("first")
    md = MyDatasetFirst(
        df_main.copy(),
        bug_localization_accuracy
    )

else:
    print("second")
    md = MyDatasetSecond(
        df_main.copy(),
        preprocessing_params["dataset"]["columns_name"], 
        preprocessing_params["dataset"]["columns_dtype"],
        bug_localization_accuracy,
        top_k=preprocessing_params["top_k"]
    )

second


In [11]:
md.select_brids_replace_cids()
md.free_memory()

In [12]:
md.df_data

{0.4:        bug_report_id  class_id  time_stamp  label
 0              27758     90276  1582785511      0
 1              27758    107081  1582785511      0
 2              27758    114937  1582785511      0
 3              27758    103283  1582785511      1
 4              27758     82681  1582785511      0
 ...              ...       ...         ...    ...
 28044          39606    114374  1435703450      0
 28045          39606    113678  1435703450      0
 28046          39606    115781  1435703450      0
 28047          39606    113169  1435703450      0
 28048          39606    118193  1435703450      0
 
 [28049 rows x 4 columns],
 0.5:        bug_report_id  class_id  time_stamp  label
 0              27760    116664  1582890005      0
 1              27760     82753  1582890005      1
 2              27760    112457  1582890005      1
 3              27760    106501  1582890005      0
 4              27760    106951  1582890005      0
 ...              ...       ...         ...

# save

In [13]:
for bla, df in md.df_data.items():
    print("-" * 10, bla , "-" * 10)
    print(df)
    print(len(df.groupby("bug_report_id")))
    print(len(df))
    
    if which_method == "first":
        path_folder = pre_path_data + "first/bla={}/".format(bla)
        my_file_name = "{} _ bla={}.csv".format(dataset_file_name, bla)
        
    else:
        path_folder = pre_path_data + "second/k={}/bla={}/".format(top_k, bla)
        my_file_name = "{} _ k={}, bla={}.csv".format(dataset_file_name, top_k, bla)
    
    
    full_path = path_folder + my_file_name
    df.to_csv(full_path, header=False, index=False)

---------- 0.4 ----------
       bug_report_id  class_id  time_stamp  label
0              27758     90276  1582785511      0
1              27758    107081  1582785511      0
2              27758    114937  1582785511      0
3              27758    103283  1582785511      1
4              27758     82681  1582785511      0
...              ...       ...         ...    ...
28044          39606    114374  1435703450      0
28045          39606    113678  1435703450      0
28046          39606    115781  1435703450      0
28047          39606    113169  1435703450      0
28048          39606    118193  1435703450      0

[28049 rows x 4 columns]
3610
28049
---------- 0.5 ----------
       bug_report_id  class_id  time_stamp  label
0              27760    116664  1582890005      0
1              27760     82753  1582890005      1
2              27760    112457  1582890005      1
3              27760    106501  1582890005      0
4              27760    106951  1582890005      0
...        